# 🌍 Demo 5: Ground Truth Challenge – Validating Models with **Your** Data

In this demo, you will evaluate an AI weather forecast **against your own local ground truth data** — not just ERA5. This reveals whether the model performs well in **your region of interest**, and teaches a key lesson:

> ❝A model trained on global reanalysis (like ERA5) may not work well where you live.❞

This gap is why **model localization** is essential for real-world AI deployment.

## 0) Setup

In [5]:
# If needed, install libs (uncomment):
# !pip install xarray pandas numpy matplotlib ipywidgets meteostat xskillscore

import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
from datetime import datetime, timedelta, timezone

try:
    from meteostat import Point, Hourly
    METEOSTAT_AVAILABLE = True
except Exception:
    METEOSTAT_AVAILABLE = False

try:
    import xskillscore as xs
    XSS_AVAILABLE = True
except Exception:
    XSS_AVAILABLE = False

import warnings
warnings.filterwarnings('ignore')

plt.rcParams['figure.figsize'] = (10,4)

## 1) 🔧 Select Your Forecast

In [6]:
forecast_source = widgets.RadioButtons(
    options=['My forecast from Demo 2', 'Pre-generated: GraphCast (sample)'],
    description='Forecast:',
    value='My forecast from Demo 2'
)

forecast_path = widgets.Text(
    description='Forecast NetCDF:',
    placeholder='e.g., ./demo2_output/forecast.nc',
    layout=widgets.Layout(width='80%')
)

graphcast_hint = widgets.HTML('<small>For the sample, point to a GraphCast-like NetCDF with variables similar to Demo 2.</small>')
display(forecast_source, forecast_path, graphcast_hint)

forecast_ds = None
forecast_info = widgets.Output()

def load_forecast(path):
    try:
        ds = xr.open_dataset(path)
        return ds
    except Exception as e:
        display(Markdown(f"❌ **Error loading forecast**: {e}"))
        return None

def on_load_forecast(_):
    global forecast_ds
    with forecast_info:
        forecast_info.clear_output()
        if not forecast_path.value.strip():
            display(Markdown("❌ Please enter a valid path."))
            return
        ds = load_forecast(forecast_path.value)
        if ds is None:
            return
        forecast_ds = ds
        try:
            times = pd.to_datetime(ds.time.values)
            start, end = times.min(), times.max()
            freq_guess = pd.infer_freq(times[:10]) or "unknown"
            display(Markdown(f"✅ **Forecast loaded**. Time: `{start}` → `{end}` (≈ {len(times)} steps, freq≈{freq_guess})"))
            display(Markdown(f"**Variables:** {list(ds.data_vars)}"))
            lat_name = 'latitude' if 'latitude' in ds.coords else ('lat' if 'lat' in ds.coords else None)
            lon_name = 'longitude' if 'longitude' in ds.coords else ('lon' if 'lon' in ds.coords else None)
            if not (lat_name and lon_name):
                display(Markdown("⚠️ Could not find standard `latitude/longitude` coordinates. Please check your file."))
        except Exception:
            pass

btn_load_fc = widgets.Button(description="Load Forecast", button_style='primary')
btn_load_fc.on_click(on_load_forecast)
display(btn_load_fc, forecast_info)

RadioButtons(description='Forecast:', options=('My forecast from Demo 2', 'Pre-generated: GraphCast (sample)')…

Text(value='', description='Forecast NetCDF:', layout=Layout(width='80%'), placeholder='e.g., ./demo2_output/f…

HTML(value='<small>For the sample, point to a GraphCast-like NetCDF with variables similar to Demo 2.</small>'…

Button(button_style='primary', description='Load Forecast', style=ButtonStyle())

Output()

## 2) 📥 Bring Your Own Ground Truth (or fetch real station data)

In [ ]:
obs_mode = widgets.ToggleButtons(
    options=[('Use my file (CSV/NetCDF)','file'), ('Fetch real station data (Meteostat)','fetch')],
    description='Observations:'
)

obs_path = widgets.Text(
    description='Obs path:',
    placeholder='e.g., ./data/local_stations.nc or ./data/station_data.csv',
    layout=widgets.Layout(width='80%')
)

country_dropdown = widgets.Dropdown(
    options=['Bangladesh','Chile','Nigeria','Ethiopia','Kenya'],
    description='Country:'
)
var_dropdown = widgets.Dropdown(
    options=[('2m temperature (2t)','2t'), ('Total precip (tp)','tp')],
    description='Variable:'
)
custom_coords = widgets.Text(
    description='lat,lon:',
    placeholder='Optional override, e.g., 23.78,90.40 (Dhaka)',
    layout=widgets.Layout(width='60%')
)

display(obs_mode, obs_path, country_dropdown, var_dropdown, custom_coords)
obs_ds = None
obs_info = widgets.Output()

COUNTRY_DEFAULT_COORDS = {
    "Bangladesh": (23.8103, 90.4125),
    "Chile": (-33.4489, -70.6693),
    "Nigeria": (6.5244, 3.3792),
    "Ethiopia": (9.0039, 38.7468),
    "Kenya": (-1.2921, 36.8219)
}

def _parse_latlon(text):
    try:
        lat_str, lon_str = text.split(",")
        return float(lat_str.strip()), float(lon_str.strip())
    except Exception:
        return None

def load_observations_file(path):
    try:
        if path.endswith('.csv'):
            df = pd.read_csv(path, parse_dates=['time'])
            if not {'time','lat','lon'}.issubset(df.columns):
                raise ValueError("CSV must contain 'time','lat','lon' plus at least one variable column.")
            obs = df.set_index(['time','lat','lon']).to_xarray()
        else:
            obs = xr.open_dataset(path)
        return obs
    except Exception as e:
        display(Markdown(f"❌ **Error loading observations**: {e}"))
        return None

def fetch_observations_from_meteostat(country, var_key, coords_override=None, forecast_like_ds=None):
    if not METEOSTAT_AVAILABLE:
        raise RuntimeError("`meteostat` not available. Install with `pip install meteostat`.")
    if forecast_like_ds is None:
        raise RuntimeError("Load the forecast first so we can align time range.")

    times = pd.to_datetime(forecast_like_ds.time.values).tz_localize('UTC')
    start_time, end_time = times.min().to_pydatetime(), times.max().to_pydatetime()

    if coords_override:
        latlon = _parse_latlon(coords_override)
        if latlon is None:
            raise ValueError("Invalid custom coords format. Use 'lat,lon'.")
        lat, lon = latlon
    else:
        lat, lon = COUNTRY_DEFAULT_COORDS[country]

    loc = Point(lat, lon)
    df = Hourly(loc, start_time, end_time).fetch()
    if df.empty:
        raise RuntimeError(f"No station data for {country} at coords {lat},{lon} for the requested period.")

    if var_key.lower() == '2t':
        if 'temp' not in df.columns:
            raise RuntimeError("Meteostat did not return 'temp' (°C).")
        series = df['temp'] + 273.15  # Kelvin
        out_name = 'temperature'; agg = 'mean'
    elif var_key.lower() == 'tp':
        if 'prcp' not in df.columns:
            raise RuntimeError("Meteostat did not return 'prcp' (mm).")
        series = df['prcp'] / 1000.0  # meters
        out_name = 'precipitation'; agg = 'sum'
    else:
        raise ValueError("var_key must be '2t' or 'tp'.")

    if agg == 'mean':
        s6 = series.resample('6H').mean()
    else:
        s6 = series.resample('6H').sum()

    sixhour_index = pd.date_range(times.min(), times.max(), freq='6H', tz='UTC')
    s6 = s6.reindex(sixhour_index)

    out_df = pd.DataFrame({
        'time': s6.index.tz_convert(None),
        'lat': lat,
        'lon': lon,
        out_name: s6.values
    }).dropna(subset=[out_name])

    return out_df

def on_load_obs(_):
    global obs_ds
    with obs_info:
        obs_info.clear_output()
        mode = obs_mode.value

        if mode == 'file':
            if not obs_path.value.strip():
                display(Markdown("❌ Please provide a path to your CSV/NetCDF observations."))
                return
            obs = load_observations_file(obs_path.value)
            if obs is None:
                return
            obs_ds = obs
            display(Markdown("✅ **Loaded your observation file.**"))
            display(Markdown(f"**Variables:** {list(obs_ds.data_vars)}"))
        else:
            if forecast_ds is None:
                display(Markdown("❌ Load a forecast first."))
                return
            try:
                df = fetch_observations_from_meteostat(
                    country_dropdown.value, var_dropdown.value, custom_coords.value.strip() or None, forecast_like_ds=forecast_ds
                )
                fname = f"local_obs_{country_dropdown.value}_{var_dropdown.value}.csv".replace(" ", "_")
                df.to_csv(fname, index=False)
                display(Markdown(f"✅ **Fetched real station data** for **{country_dropdown.value}** ({var_dropdown.value}). Saved as `{fname}`."))
                obs_ds = df.set_index(['time','lat','lon']).to_xarray()
                display(df.head())
            except Exception as e:
                display(Markdown(f"❌ {e}"))

btn_load_obs = widgets.Button(description="Load / Fetch Observations", button_style='primary')
btn_load_obs.on_click(on_load_obs)
display(btn_load_obs, obs_info)

ToggleButtons(description='Observations:', options=(('Use my file (CSV/NetCDF)', 'file'), ('Fetch real station…

Text(value='', description='Obs path:', layout=Layout(width='80%'), placeholder='e.g., ./data/local_stations.n…

Dropdown(description='Country:', options=('Bangladesh', 'Chile', 'Nigeria', 'Ethiopia', 'Kenya'), value='Bangl…

Dropdown(description='Variable:', options=(('2m temperature (2t)', '2t'), ('Total precip (tp)', 'tp')), value=…

Text(value='', description='lat,lon:', layout=Layout(width='60%'), placeholder='Optional override, e.g., 23.78…

Button(button_style='primary', description='Load / Fetch Observations', style=ButtonStyle())

Output()

## 3) 🔗 Align Data: Interpolate Forecast to Station Locations

In [8]:
# === Align forecast to observation points (robust to time+step/valid_time & timezones) ===
align_info = widgets.Output()

def _coord_names(ds):
    lat_name = 'latitude' if 'latitude' in ds.coords else ('lat' if 'lat' in ds.coords else None)
    lon_name = 'longitude' if 'longitude' in ds.coords else ('lon' if 'lon' in ds.coords else None)
    return lat_name, lon_name

def _pick_fc_var(ds):
    # Prefer common names if present; otherwise first data var
    preferred = ['2t', 'tp', 't2m', 'temperature', 'precipitation']
    for k in preferred:
        if k in ds.data_vars:
            return k
    return list(ds.data_vars)[0]

def _valid_times_and_axis(ds):
    """Return (valid_times_as_pandas, axis_name in {'valid_time','step','time'})"""
    # 1) explicit valid_time
    if 'valid_time' in ds.coords:
        vt = pd.to_datetime(ds['valid_time'].values)
        return vt, 'valid_time'
    # 2) init time + forecast step
    if 'time' in ds.coords and ('step' in ds.coords or 'step' in ds.dims):
        base = pd.to_datetime(ds['time'].values)
        base0 = pd.to_datetime(base[0])
        step_vals = ds['step'].values
        try:
            step_td = pd.to_timedelta(step_vals)
        except Exception:
            step_td = pd.to_timedelta(step_vals, unit='h')
        vt = base0 + step_td
        return pd.to_datetime(vt), 'step'
    # 3) plain time axis
    if 'time' in ds.coords:
        vt = pd.to_datetime(ds['time'].values)
        return vt, 'time'
    raise ValueError("Could not identify a time axis (time/step/valid_time).")

def interpolate_to_points(forecast_ds, obs_ds):
    lat_name, lon_name = _coord_names(forecast_ds)
    if not (lat_name and lon_name):
        raise ValueError("Could not find 'latitude'/'longitude' coordinates in forecast dataset.")

    fc_var = _pick_fc_var(forecast_ds)
    obs_var = list(obs_ds.data_vars)[0]

    # Observation times -> make naive UTC
    obs_times = pd.to_datetime(obs_ds.time.values)
    if getattr(obs_times, 'tz', None) is not None:
        obs_times = obs_times.tz_convert('UTC').tz_localize(None)
    else:
        # treat as UTC-naive
        obs_times = pd.to_datetime(obs_times)

    # Forecast valid times (make naive UTC if needed)
    vt, axis_name = _valid_times_and_axis(forecast_ds)
    if getattr(vt, 'tz', None) is not None:
        vt = vt.tz_convert('UTC').tz_localize(None)
    else:
        vt = pd.to_datetime(vt)

    vt_np = np.array(vt, dtype='datetime64[ns]')

    aligned = []
    misses = 0

    for t in obs_times:
        try:
            ti = np.datetime64(pd.to_datetime(t))
            i = int(np.argmin(np.abs(vt_np - ti)))
            if axis_name == 'valid_time':
                time_sel = {'valid_time': vt[i]}
            elif axis_name == 'step':
                time_sel = {'step': forecast_ds['step'].values[i]}
            else:
                time_sel = {'time': vt[i]}

            obs_slice = obs_ds.sel(time=t)
            lats = np.atleast_1d(obs_slice.lat.values)
            lons = np.atleast_1d(obs_slice.lon.values)

            for lat in lats:
                for lon in lons:
                    # wrap to 0–360 if needed
                    max_lon = float(forecast_ds[lon_name].max())
                    lon_wrapped = float(lon) % 360 if max_lon > 180 else float(lon)

                    obs_val = float(obs_ds.sel(time=t, lat=lat, lon=lon)[obs_var].values)
                    fc_point = forecast_ds[fc_var].sel(
                        **time_sel,
                        **{lat_name: float(lat), lon_name: lon_wrapped},
                        method='nearest'
                    )
                    # .item() may fail on masked; use float(np.asarray(...))
                    fc_val = float(np.asarray(fc_point))
                    aligned.append({
                        'time': pd.to_datetime(t),
                        'forecast': fc_val,
                        'observation': obs_val,
                        'lat': float(lat),
                        'lon': float(lon)
                    })
        except Exception:
            misses += 1
            continue

    df = pd.DataFrame(aligned).dropna()
    if df.empty and misses > 0:
        print(f"(note) skipped {misses} points due to time/coord mismatches.")
    return df

results_df = None

def on_align(_):
    global results_df
    with align_info:
        align_info.clear_output()
        if forecast_ds is None or obs_ds is None:
            display(Markdown("❌ Load both **forecast** and **observations** first."))
            return
        display(Markdown("⏳ Aligning forecast to observation points..."))
        results_df = interpolate_to_points(forecast_ds, obs_ds)
        if results_df.empty:
            display(Markdown("❌ No overlaps found. Check time axis (time/step/valid_time) and coordinates."))
            # quick debug print
            try:
                vt, axis_name = _valid_times_and_axis(forecast_ds)
                display(Markdown(f"- Detected forecast axis: **{axis_name}** with {len(vt)} entries."))
                display(Markdown(f"- Obs sample times: **{pd.to_datetime(obs_ds.time.values)[:3].tolist()}**"))
            except Exception as e:
                display(Markdown(f"(debug) {e}"))
        else:
            display(Markdown(f"✅ Matched **{len(results_df)}** time-point pairs."))
            display(results_df.head())

btn_align = widgets.Button(description="Align & Preview", button_style='primary')
btn_align.on_click(on_align)
display(btn_align, align_info)


Button(button_style='primary', description='Align & Preview', style=ButtonStyle())

Output()

## 4) 📊 Compute Metrics vs Local Data

In [ ]:
metrics_out = widgets.Output()

def compute_and_plot_metrics(df):
    forecast = df['forecast'].astype(float)
    obs = df['observation'].astype(float)

    rmse = float(np.sqrt(np.mean((forecast - obs)**2)))
    bias = float(np.mean(forecast - obs))
    corr = float(np.corrcoef(forecast, obs)[0,1]) if len(df) > 1 else np.nan

    display(Markdown("### 📊 Model Performance vs **Your Local Data**"))
    display(Markdown(f"- **RMSE**: {rmse:.3f}"))
    display(Markdown(f"- **Bias**: {bias:.3f}"))
    display(Markdown(f"- **Correlation**: {corr:.3f}"))

    if XSS_AVAILABLE:
        try:
            import xskillscore as xs
            xr_fc = xr.DataArray(forecast.values, dims=['points'])
            xr_ob = xr.DataArray(obs.values, dims=['points'])
            mae = float(xs.mae(xr_fc, xr_ob).values)
            display(Markdown(f"- **MAE (xskillscore)**: {mae:.3f}"))
        except Exception:
            pass

    plt.figure()
    plt.plot(df['time'], df['observation'], label='Observation', marker='o')
    plt.plot(df['time'], df['forecast'], label='Forecast', marker='x')
    plt.xlabel('Time'); plt.ylabel('Value'); plt.title('Forecast vs Local Observations')
    plt.legend(); plt.xticks(rotation=45); plt.grid(alpha=0.3); plt.tight_layout(); plt.show()

    return rmse

def on_metrics(_):
    global last_rmse_local
    with metrics_out:
        metrics_out.clear_output()
        if results_df is None or results_df.empty:
            display(Markdown("❌ No aligned data. Run alignment first."))
            return
        last_rmse_local = compute_and_plot_metrics(results_df)

btn_metrics = widgets.Button(description="Compute Metrics", button_style='primary')
btn_metrics.on_click(on_metrics)
display(btn_metrics, metrics_out)

Button(button_style='primary', description='Compute Metrics', style=ButtonStyle())

Output()

## 5) 🔍 Compare with Demo 4 (vs ERA5)

In [ ]:
demo4_rmse = widgets.FloatText(description="Your RMSE vs ERA5:", value=2.0, step=0.1)
cmp_out = widgets.Output()

def on_compare(_):
    with cmp_out:
        cmp_out.clear_output()
        if 'last_rmse_local' not in globals():
            if results_df is None or results_df.empty:
                display(Markdown("❌ Compute metrics first."))
                return
            rmse_local = float(np.sqrt(np.mean((results_df['forecast'] - results_df['observation'])**2)))
        else:
            rmse_local = last_rmse_local

        rmse_era5 = demo4_rmse.value
        display(Markdown("### 🔄 Comparison: Local vs Reanalysis"))
        display(Markdown(f"| Source | RMSE |\n|---|---|\n| ERA5 (Demo 4) | {rmse_era5:.3f} |\n| **Your Data** | **{rmse_local:.3f}** |"))

        if rmse_local > rmse_era5:
            display(Markdown("⚠️ The model performs **worse** against your real-world data than against ERA5."))
            display(Markdown("This suggests the model is optimized for ERA5, not your region — a sign that **localization** is needed."))
        else:
            display(Markdown("✅ The model generalizes well to your local conditions!"))

display(demo4_rmse)
btn_compare = widgets.Button(description="Compare", button_style='primary')
btn_compare.on_click(on_compare)
display(btn_compare, cmp_out)

FloatText(value=2.0, description='Your RMSE vs ERA5:', step=0.1)

Button(button_style='primary', description='Compare', style=ButtonStyle())

Output()

## (Optional) 🗂 Generate CSVs for All Five Countries & Both Variables

This will fetch **real station data** for each country (nearest to the capital by default) for both variables:
- `2t` (2m temperature, Kelvin, aggregated by mean to 6-hourly)
- `tp` (total precipitation, meters, aggregated by sum to 6-hourly)

It uses the **forecast time window** you loaded above to align timestamps.

In [13]:
bulk_out = widgets.Output()

def generate_all_csvs(_):
    with bulk_out:
        bulk_out.clear_output()
        if forecast_ds is None:
            display(Markdown("❌ Load a forecast first."))
            return
        if not METEOSTAT_AVAILABLE:
            display(Markdown("❌ `meteostat` not available. Install with `pip install meteostat`."))
            return

        countries = ['Bangladesh','Chile','Nigeria','Ethiopia','Kenya']
        vars_ = ['2t','tp']
        created = []
        for c in countries:
            for v in vars_:
                try:
                    df = fetch_observations_from_meteostat(c, v, None, forecast_like_ds=forecast_ds)
                    fname = f"local_obs_{c}_{v}.csv".replace(" ", "_")
                    df.to_csv(fname, index=False)
                    created.append(fname)
                    display(Markdown(f"✅ Saved `{fname}` ({len(df)} rows)"))
                except Exception as e:
                    display(Markdown(f"⚠️ {c} {v}: {e}"))
        if created:
            display(Markdown("**Done.** Created files:"))
            for f in created:
                display(Markdown(f"- `{f}`"))

btn_bulk = widgets.Button(description="Generate all 10 CSVs", button_style='warning')
btn_bulk.on_click(generate_all_csvs)
display(btn_bulk, bulk_out)

Button(button_style='warning', description='Generate all 10 CSVs', style=ButtonStyle())

Output()

In [12]:
# --- PATCH: robust timezone handling for Meteostat fetch + alignment ---
def fetch_observations_from_meteostat(country, var_key, coords_override=None, forecast_like_ds=None):
    if not METEOSTAT_AVAILABLE:
        raise RuntimeError("`meteostat` not available. Install with `pip install meteostat`.")
    if forecast_like_ds is None:
        raise RuntimeError("Load the forecast first so we can align the time window.")

    # 1) Forecast times -> make tz-aware UTC
    times = pd.to_datetime(forecast_like_ds.time.values)
    if getattr(times, "tz", None) is None:
        times = times.tz_localize("UTC")
    else:
        times = times.tz_convert("UTC")
    tmin_aware = times.min()
    tmax_aware = times.max()

    # Meteostat likes naive datetimes; pass naive UTC bounds
    start_time = tmin_aware.tz_localize(None).to_pydatetime()
    end_time   = tmax_aware.tz_localize(None).to_pydatetime()

    # 2) Coordinates (default = capital)
    COUNTRY_DEFAULT_COORDS = {
        "Bangladesh": (23.8103, 90.4125),   # Dhaka
        "Chile": (-33.4489, -70.6693),      # Santiago
        "Nigeria": (6.5244, 3.3792),        # Lagos
        "Ethiopia": (9.0039, 38.7468),      # Addis Ababa
        "Kenya": (-1.2921, 36.8219)         # Nairobi
    }
    def _parse_latlon(text):
        try:
            a,b = text.split(","); return float(a), float(b)
        except Exception:
            return None

    if coords_override:
        ll = _parse_latlon(coords_override)
        if ll is None:
            raise ValueError("Invalid custom coords. Use 'lat,lon' (e.g., 23.78,90.40).")
        lat, lon = ll
    else:
        lat, lon = COUNTRY_DEFAULT_COORDS[country]

    # 3) Fetch Meteostat & force index to tz-aware UTC
    loc = Point(lat, lon)
    df = Hourly(loc, start_time, end_time).fetch()
    if df.empty:
        raise RuntimeError(f"No station data for {country} at {lat},{lon} in the requested window.")
    if df.index.tz is None:
        df.index = df.index.tz_localize("UTC")
    else:
        df.index = df.index.tz_convert("UTC")

    # 4) Map variable & units to model convention
    if var_key.lower() == "2t":
        if "temp" not in df.columns:
            raise RuntimeError("Meteostat did not return 'temp' (°C).")
        series = df["temp"] + 273.15     # Kelvin
        out_name = "temperature"
        agg = "mean"
    elif var_key.lower() == "tp":
        if "prcp" not in df.columns:
            raise RuntimeError("Meteostat did not return 'prcp' (mm).")
        series = df["prcp"] / 1000.0     # meters
        out_name = "precipitation"
        agg = "sum"
    else:
        raise ValueError("var_key must be '2t' or 'tp'.")

    # 5) Resample to 6-hour cadence in UTC (tz-aware)
    s6 = series.resample("6H").mean() if agg == "mean" else series.resample("6H").sum()

    # 6) Align exactly to the forecast timestamps (also tz-aware)
    sixhour_index = pd.date_range(tmin_aware, tmax_aware, freq="6H", tz="UTC")
    s6 = s6.reindex(sixhour_index)

    # 7) Output tz-naive times for the CSV (notebook expects naive)
    out_df = pd.DataFrame({
        "time": s6.index.tz_convert(None),  # drop tz info for CSV
        "lat": lat,
        "lon": lon,
        out_name: s6.values
    }).dropna(subset=[out_name])

    return out_df

print("Patched fetch_observations_from_meteostat() with robust timezone handling.")


Patched fetch_observations_from_meteostat() with robust timezone handling.


## 6) 🌐 Key Takeaway: The Need for Localization

You've just uncovered a critical insight in AI weather modeling:

> ❝Models are trained on **reanalysis data (like ERA5)** — not real observations.❞

If the model performs poorly on **your local station data**, it doesn't mean the model is bad — it means:
- ERA5 may not represent your region accurately (e.g., complex terrain, data gaps).
- The model learned to predict **ERA5 patterns**, not **your weather**.

**Localization** bridges this: fine-tune with local data, apply bias correction/post-processing, or use regional adapters.